## Resources    
* [Midi Tutorial](https://www.cs.cmu.edu/~music/cmsip/readings/MIDI%20tutorial%20for%20programmers.html#:~:text=Be%20cautious%20however%20about%20the,very%20low%20note%20will%20result.) 
* [Synthesizer to Play MIDI Files](https://onlinesequencer.net/import2/5d20579db21d61f35db35c2c988d9774?title=HCal_Test.mid)
* [MIDIUtils Docs](https://midiutil.readthedocs.io/en/1.2.1/common.html)
* [Jupyter Notebook Markdown/LaTeX Resources](https://towardsdatascience.com/write-markdown-latex-in-the-jupyter-notebook-10985edb91fd)


In [140]:
from keras.utils import np_utils #Utilities to transform data.
#from sklearn.preprocessing import normalize

In [80]:
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import Divider, Size
from mpl_toolkits.axes_grid1.mpl_axes import Axes
import numpy as np
import ROOT
import sys
from midiutil import MIDIFile

In [65]:
data = "/home/skbarcus/Projects/Music/HCal/Data/"

infile = "/home/skbarcus/JLab/SBS/HCal/Analysis/Cosmics/rootfiles/fadc_f1tdc_820.root"
#infile = sys.argv[1]

print("Reading from", infile)

inFile = ROOT.TFile.Open(infile," READ ")

tree = inFile.Get("T")
print(tree.GetEntries())

Reading from /home/skbarcus/JLab/SBS/HCal/Analysis/Cosmics/rootfiles/fadc_f1tdc_820.root
306807


In [143]:
read_all = 0
nevts = 50 #run 820 25 events was a nice sample.
adc_samps_vals = []
adc_vals = []
tdc_vals = []
hit_vals = []

min_note = 0
max_note = 115

min_adc_int_hit = 7000
max_adc_int = 0

if read_all == 1:
    loop = tree.GetEntries()
else:
    loop = nevts

tree.GetEntry(0)

fadc_window = getattr(tree,"sbs.hcal.nsamps") #Reads in fADC window widths.
fadc_window = fadc_window[0]
print('fADC window = ',fadc_window)

channels = getattr(tree,"sbs.hcal.a") #Reads in all individual fADC samples for a whole event.
channels = len(channels)
print('channels = ',channels)

#Loop over entries.
for entryNum in range (0, loop):
    tree.GetEntry(entryNum)
    adc_samps_evt = getattr(tree,"sbs.hcal.samps") #Reads in all individual fADC samples for a whole event.
    #print('adc_samps_evt = ',adc_samps_evt)
    adc_evt = getattr(tree,"sbs.hcal.a") #Reads in all fADC integrals for a whole event.
    adc_vals.append([])
    for pmt in range(0,len(adc_evt)-2):
        if adc_evt[pmt]>max_adc_int:
            max_adc_int = adc_evt[pmt]
    #print('adc_vals shape = ', adc_vals)
    tdc_evt = getattr(tree,"sbs.hcal.tdc")
    
    for pmt in range(0,len(adc_evt)):
        adc_vals[entryNum].append(adc_evt[pmt])
 
print('max_adc_int = ',max_adc_int)

adc_arr = np.array(adc_vals)
print('adc_arr shape = ',adc_arr.shape)  
#print('adc_arr = ',adc_arr)  


fADC window =  20.0
channels =  144
max_adc_int =  29215.0
adc_arr shape =  (50, 144)


In [144]:
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 120   # In BPM
volume   = 100  # 0-127, as per the MIDI standard
nadc_int_notes = 0

MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created automatically)
MyMIDI.addTempo(track, time, tempo)

print('nevts = ',adc_arr.shape[0],', nPMTs = ',adc_arr.shape[1])
for evt in range(0,adc_arr.shape[0]):
    for pmt in range(0,adc_arr.shape[1]-2):
        if adc_arr[evt][pmt] > min_adc_int_hit:
            nadc_int_notes = nadc_int_notes + 1
            adc_int_note = int((adc_arr[evt][pmt] - min_adc_int_hit)/(max_adc_int - min_adc_int_hit) * max_note)
            print('adc_int_note = ',adc_arr[evt][pmt],' = adc_int_note = ', adc_int_note)
            MyMIDI.addNote(track, channel, adc_int_note, time + nadc_int_notes, duration, volume)
            #Seems to be an octave issue (adds or subtracts 12 from value) limiting to 115 instead of 127.

print('Total number of notes = ', nadc_int_notes)
with open("HCal_Test.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

nevts =  50 , nPMTs =  144
adc_int_note =  8016.0  = adc_int_note =  5
adc_int_note =  7729.0  = adc_int_note =  3
adc_int_note =  16731.0  = adc_int_note =  50
adc_int_note =  7694.0  = adc_int_note =  3
adc_int_note =  17340.0  = adc_int_note =  53
adc_int_note =  23116.0  = adc_int_note =  83
adc_int_note =  11604.0  = adc_int_note =  23
adc_int_note =  7486.0  = adc_int_note =  2
adc_int_note =  18287.0  = adc_int_note =  58
adc_int_note =  15843.0  = adc_int_note =  45
adc_int_note =  18136.0  = adc_int_note =  57
adc_int_note =  26282.0  = adc_int_note =  99
adc_int_note =  7136.0  = adc_int_note =  0
adc_int_note =  7121.0  = adc_int_note =  0
adc_int_note =  9318.0  = adc_int_note =  11
adc_int_note =  7180.0  = adc_int_note =  0
adc_int_note =  14426.0  = adc_int_note =  38
adc_int_note =  7330.0  = adc_int_note =  1
adc_int_note =  7744.0  = adc_int_note =  3
adc_int_note =  10690.0  = adc_int_note =  19
adc_int_note =  10749.0  = adc_int_note =  19
adc_int_note =  7009.0  = 